In [2]:
import numpy as np
import random
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import CausalInference

# 1. Define a model where context matters
model = BayesianNetwork([
    ('last_tool_change', 'machine_state'),
    ('machine_state', 'relative_processing_time_deviation'),
    ('machine_state', 'cleaning'),
    ('cleaning', 'relative_processing_time_deviation')
])

# Update CPD for 'last_tool_change' to have 5 states
cpd_last_tool_change = TabularCPD(
    variable='last_tool_change', 
    variable_card=5,  # 5 states: 0, 1, 2, 3, 4
    values=[
        [0.2],  # P(last_tool_change=0)
        [0.2],  # P(last_tool_change=1)
        [0.2],  # P(last_tool_change=2)
        [0.2],  # P(last_tool_change=3)
        [0.2]   # P(last_tool_change=4)
    ]
)

# Update CPD for 'machine_state' to account for 5 states of 'last_tool_change'
cpd_machine_state = TabularCPD(
    variable='machine_state', 
    variable_card=2,  # 2 states: 0 and 1
    values=[
    [0.6, 0.4, 0.3, 0.4, 0.2],  # P(machine_state=0 | last_tool_change)
    [0.4, 0.6, 0.7, 0.6, 0.8]   # P(machine_state=1 | last_tool_change)
    ],
    evidence=['last_tool_change'],  # Parent node
    evidence_card=[5]  # Number of states of parent
)

cpd_cleaning = TabularCPD(
    variable='cleaning', 
    variable_card=2,  # 2 states: 0 and 1
    values=[
        [0.5, 0.2],  # P(cleaning=0 | machine_state)
        [0.5, 0.8]   # P(cleaning=1 | machine_state)
    ],
    evidence=['machine_state'],  # Parent node
    evidence_card=[2]  # Number of states of parent
)

# Adjust CPD for 'relative_processing_time_deviation' to make cleaning impactful
cpd_relative_processing_time_deviation = TabularCPD(
    variable='relative_processing_time_deviation', 
    variable_card=3,  # 3 states: 0.9, 1.0, 1.2
    values=[
        [0.9, 0.3, 0.2, 0.8],  # P(0.9)
        [0.1, 0.6, 0.7, 0.15],  # P(1.0)
        [0.0, 0.1, 0.1, 0.05],  # P(1.2)
    ],
    evidence=['machine_state', 'cleaning'],
    evidence_card=[2, 2]
)
model.add_cpds(cpd_last_tool_change, cpd_machine_state, cpd_cleaning, cpd_relative_processing_time_deviation)
assert model.check_model()
ci = CausalInference(model)
deviation_map = {0: 0.9, 1: 1.0, 2: 1.2}

# 2. Simulate a sequence of operations
n_ops = 500
np.random.seed(77)
last_tool_changes = np.random.choice([0, 1], size=n_ops)
base_duration = 10

cbn_times = []
always_clean_times = []
never_clean_times = []

for ltc in last_tool_changes:
    base_duration = random.choice([10, 20, 30])
    # CBN-based: choose cleaning if E[deviation|do(cleaning=1)] < E[deviation|do(cleaning=0)]
    factor_clean = ci.query(['relative_processing_time_deviation'], evidence={'last_tool_change': ltc}, do={'cleaning': 1},show_progress=False)
    factor_no_clean = ci.query(['relative_processing_time_deviation'], evidence={'last_tool_change': ltc}, do={'cleaning': 0},show_progress=False)
    exp_clean = sum(factor_clean.values[i] * deviation_map[i] for i in range(3))
    exp_no_clean = sum(factor_no_clean.values[i] * deviation_map[i] for i in range(3))
    if exp_clean < exp_no_clean:
        cbn_times.append(base_duration * exp_clean)
    else:
        cbn_times.append(base_duration * exp_no_clean)
    # Baselines
    always_clean_times.append(base_duration * exp_clean)
    never_clean_times.append(base_duration * exp_no_clean)

# 3. Compare results
print(f"Mean CBN-based: {np.mean(cbn_times):.3f}")
print(f"Mean Always clean: {np.mean(always_clean_times):.3f}")
print(f"Mean Never clean: {np.mean(never_clean_times):.3f}")

Mean CBN-based: 18.694
Mean Always clean: 18.896
Mean Never clean: 18.790
